# Autómatos Finitos Determinísticos

## Conjuntos em Python

In [ ]:
print ({1,2,3} <= {3,2,4})
print ({1,2,3} <= {4,1,2,3})
print ({1,2,3} | {4,1,3})
print ({1,2,3} & {1,3,4})
print (set([1,2,3,2,1]))
print ({[1,2],[2,3],[3,4]})      # não é possível construir conjuntos de tipos mutáveis!

False
True
{1, 2, 3, 4}
{1, 3}
{1, 2, 3}


TypeError: ignored

In [ ]:
for x in set({1,2,3,4,3,2,1}) : 
  print (x)

1
2
3
4


### Definição de conjuntos por compreensão
Dialecto muito útil! Será utilizado em algumas funções sobre autómatos

In [ ]:
s1 = { (x,y) for x in range(1,10) for y in {"aa","bb","cc"} }
print(len(s1))
print(s1)

s2 = { (x,y) for x in range(1,11) for y in range(1,11) if x!=y }
print(len(s2))
print(s2)



27
{(4, 'bb'), (3, 'cc'), (5, 'aa'), (6, 'aa'), (8, 'aa'), (7, 'cc'), (8, 'bb'), (3, 'bb'), (9, 'cc'), (4, 'aa'), (2, 'bb'), (1, 'cc'), (2, 'aa'), (5, 'cc'), (6, 'cc'), (1, 'bb'), (7, 'bb'), (8, 'cc'), (6, 'bb'), (3, 'aa'), (4, 'cc'), (7, 'aa'), (1, 'aa'), (5, 'bb'), (9, 'bb'), (9, 'aa'), (2, 'cc')}
90
{(7, 3), (6, 9), (1, 6), (3, 7), (2, 5), (8, 5), (5, 8), (10, 8), (6, 7), (10, 7), (7, 6), (6, 10), (4, 10), (3, 2), (2, 6), (8, 2), (4, 5), (9, 3), (7, 5), (3, 1), (7, 8), (2, 1), (8, 9), (9, 4), (5, 1), (10, 3), (7, 2), (1, 5), (3, 6), (1, 10), (8, 6), (4, 1), (10, 9), (9, 7), (6, 4), (5, 4), (10, 4), (7, 1), (3, 5), (2, 7), (8, 3), (5, 10), (4, 6), (9, 2), (6, 1), (5, 7), (7, 4), (1, 3), (4, 8), (2, 8), (9, 8), (6, 2), (3, 10), (8, 10), (1, 4), (3, 9), (2, 3), (1, 9), (8, 7), (4, 2), (9, 6), (6, 5), (5, 3), (10, 5), (6, 8), (1, 7), (3, 4), (2, 4), (8, 4), (5, 9), (4, 7), (9, 1), (5, 6), (10, 6), (1, 2), (4, 9), (2, 9), (8, 1), (6, 3), (7, 10), (2, 10), (9, 10), (10, 1), (7, 9), (3, 8)

## Representação de Autómatos
Relembre-se que um DFA é um tuplo $(Q, \Sigma, \delta, q_0,F)$. Este tuplo será representado por um dicionário com 5 campos, como ilustrado pelo autómato seguinte, que reconhece palavras com um número de 0s múltiplo de 3


In [ ]:
dfa0mult3 = { 'Q': {'IF', 'A', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('IF', '0'): 'A',
                         ('IF', '1'): 'IF',
                         ('A', '0'): 'B',
                         ('A', '1'): 'A',
                         ('B', '0'): 'IF',
                         ('B', '1'): 'B' },
              'q0': 'IF', 
              'F': {'IF'}   
            }


Note que é perfeitamente possível definir-se autómatos como dicionários que não encaixam na definição matemática de autómato. Por exemplo, 
* a função de transição pode referir estados que não tenham sido listados no conjunto de estados; ou
* a função de transição pode não ser total! Isto é, podem não estar especificadas todas as transições em todos os estados. 

Podemos ver qual o efeito destas incoerências na visualização, alterando o autómato anterior: no primeiro caso obterá um erro, e no segundo será impresso um pseudo-autómato, com função de transição parcial. 

É muito conveniente definir funções (predicados) para testar a coerência de autómatos. A primeira função em baixo admite que a função de transição possa ser parcial; a segunda obriga a que seja total.




In [ ]:
def partconsist_dfa(D):

    Q     = D["Q"]
    Sigma = D["Sigma"]
    Delta = D["Delta"]
    q0    = D["q0"]
    F     = D["F"]

    dom_Delta   = set(Delta.keys())
    range_Delta = set(Delta.values())
  
    return (Q != {}              and
            Sigma != {}          and
            dom_Delta <= { (x,y) for x in Q for y in Sigma } and
            range_Delta != {}    and
            range_Delta <= Q     and
            q0 in Q              and
            F <= Q)  

print(partconsist_dfa(dfa0mult3))


True


In [ ]:
def consist_dfa(D):

    Q     = D["Q"]
    Sigma = D["Sigma"]
    Delta = D["Delta"]

    return (partconsist_dfa(D) and
            set((Delta).keys()) == set({ (x,y) for x in Q for y in Sigma } ))  


print(consist_dfa(dfa0mult3))

True


## Execução de um Autómato e Aceitação de Palavras

Agora que temos uma representação para DFAs, queremos definir um predicado que decide se um autómato aceita ou não uma dada palavra. Para isso será necessário definir a função $\hat{\delta}$. Relembremos:

$\hat{\delta}(q, \varepsilon)\ \ \ =\ q$

$\hat{\delta}(q, xs)\ = \hat{\delta}(\delta(q,x),s)$

O autómato aceitará a palavra $s$ se $ \hat{\delta}(q_0, s) \in F$

Definiremos as funções `run_dfa` e `accepts_dfa` que implementam esta funcionalidade 

#### Versão recursiva 

Será útil o seguinte operador que descarta o primeiro elemento de uma lista:


In [ ]:
s = "gato negro"
print (s[1:])

ato negro


In [ ]:
def run_dfa_aux(D, q, s):
    if s == "" : 
        return q
    else :
        return run_dfa_aux(D, D["Delta"][(q,s[0])], s[1:])
    
def run_dfa_rec(D, s):
    return run_dfa_aux(D, D["q0"], s)


In [ ]:
print("run_dfa_rec(dfa0mult3, '101001') = ", 
          run_dfa_rec(dfa0mult3, '101001'))
print("run_dfa_rec(dfa0mult3, '101000') = ", 
          run_dfa_rec(dfa0mult3, '101000'))

run_dfa_rec(dfa0mult3, '101001') =  IF
run_dfa_rec(dfa0mult3, '101000') =  A


#### Versão iterativa 


In [ ]:
def run_dfa(D, s):
    state = D["q0"]
    
    # while s != "":
    #     state = D["Delta"][(state,s[0])]
    #     s = s[1:]
      
    for c in s : 
        state = D["Delta"][(state,c)]
        
    return state

In [ ]:
print("run_dfa(dfa0mult3, '101001') = ", 
          run_dfa(dfa0mult3, '101001'))
print("run_dfa(dfa0mult3, '101000') = ", 
          run_dfa(dfa0mult3, '101000'))

run_dfa(dfa0mult3, '101001') =  IF
run_dfa(dfa0mult3, '101000') =  A


#### Aceitação de palavras

In [ ]:
def accepts_dfa(D, s):

    assert(consist_dfa(D)), "D is not consistent."
    
    # return run_dfa_rec(D, s) in D["F"]
    return run_dfa(D, s) in D["F"]


print("accepts_dfa(dfa0mult3, '101001') = ", 
          accepts_dfa(dfa0mult3, '101001')) 
print("accepts_dfa(dfa0mult3, '101000') = ", 
          accepts_dfa(dfa0mult3, '101000')) 

accepts_dfa(dfa0mult3, '101001') =  True
accepts_dfa(dfa0mult3, '101000') =  False


### Função de aceitação com cálculo dos estados intermédios 

In [ ]:
def accepts_df (D, q, s) :

    assert(consist_dfa(D)), "D is not consistent."

    if s == "" and q in D["F"]:
        return (True, [])

    if s == "" : 
        return (False, [])
    
    nextq = D["Delta"][(q,s[0])]

    (done, path) = accepts_df(D, nextq, s[1:])
    if done : return (True, [nextq]+path)
            
    return (False, [])
            
    

def accepts_withpath (D, s) : 
    return (accepts_df(D, D["q0"], s))


print("accepts_withpath(dfa0mult3, '101001') = ", 
          accepts_withpath(dfa0mult3, '101001')) 
print("accepts_withpath(dfa0mult3, '101000') = ", 
          accepts_withpath(dfa0mult3, '101000')) 

accepts_withpath(dfa0mult3, '101001') =  (True, ['IF', 'A', 'A', 'B', 'IF', 'IF'])
accepts_withpath(dfa0mult3, '101000') =  (False, [])


## União e Intersecção de Autómatos

Vimos nas  aulas teóricas que é útil construir linguagens utilizando as operações de *união*, *intersecção*, e *complemento* de conjuntos.

É pois útil dispor também destas operações ao nível dos autómatos. Por exemplo, a união de dois autómatos D1 e D2 é ainda um autómato, que reconhece a união das linguagens reconhecidas por D1 e por D2.

A união e a intersecção de autómatos D1 e D2 são ambos autómatos cujos estados correspondem a pares de estados, sendo o primeito um estado de D1 e o segundo um estado de D2. As transições são obtidas da seguinte forma: se temos em D1 a transição A1-c->B1 e em D2 A2-c->B2, então teremos na união e na intersecção de D1 e D2 a transição (A1,A2)-c->(B1,B2).

Os dois autómatos diferem apenas nos estados finais: na união basta que um dos estados do par seja final, enquanto que na intersecção ambas as componentes terão de ser estados finais dos autómatos respectivos.

In [ ]:
def union_dfa(D1,D2) : 
    assert(consist_dfa(D1)), "D1 is not consistent."
    assert(consist_dfa(D2)), "D2 is not consistent."
    assert(D1["Sigma"] == D2["Sigma"]), "D1 and D2 have different alphabets."


    Q = { q1+"_"+q2 for q1 in D1["Q"] for q2 in D2["Q"] }
    Sigma = D1["Sigma"]
    
    Delta = {}
    for q1 in D1["Q"] :
        for q2 in D2["Q"] :
            for c in Sigma : 
                Delta[(q1+"_"+q2,c)] = D1["Delta"][(q1,c)]+"_"+D2["Delta"][(q2,c)]
    
    return { "Q"     : Q, 
             "Sigma" : Sigma ,    
             "Delta" : Delta,
             "q0"    : D1["q0"]+"_"+D2["q0"],          
#  INTERSECÇÃO           "F"     : { q1+"_"+q2 for q1 in D1["F"] for q2 in D2["F"] }
             "F"     : { q1+"_"+q2 for q1 in D1["F"] for q2 in D2["Q"] } | 
                       { q1+"_"+q2 for q1 in D1["Q"] for q2 in D2["F"] }

           }



In [ ]:
dfa1par = { 'Q': {'A', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('A', '0'): 'A',
                         ('A', '1'): 'B',
                         ('B', '0'): 'B',
                         ('B', '1'): 'A' },
              'q0': 'A', 
              'F': {'A'}   
            }

dfa0par = { 'Q': {'A', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('A', '1'): 'A',
                         ('A', '0'): 'B',
                         ('B', '1'): 'B',
                         ('B', '0'): 'A' },
              'q0': 'A', 
              'F': {'A'}   
            }

dfa0ou1par = union_dfa(dfa1par, dfa0par)
print(dfa0ou1par)


{'Q': {'A_A', 'A_B', 'B_A', 'B_B'}, 'Sigma': {'1', '0'}, 'Delta': {('B_B', '1'): 'A_B', ('B_B', '0'): 'B_A', ('B_A', '1'): 'A_A', ('B_A', '0'): 'B_B', ('A_B', '1'): 'B_B', ('A_B', '0'): 'A_A', ('A_A', '1'): 'B_A', ('A_A', '0'): 'A_B'}, 'q0': 'A_A', 'F': {'A_A', 'A_B', 'B_A'}}


In [ ]:
print("accepts_dfa(dfa1par, '10100') = ", 
          accepts_dfa(dfa1par, '10100'))
print("accepts_dfa(dfa0par, '10100') = ", 
          accepts_dfa(dfa0par, '10100'))
print("accepts_dfa(dfa0ou1par, '10100') = ", 
          accepts_dfa(dfa0ou1par, '1010'))
print()
print("accepts_dfa(dfa1par, '101001') = ", 
          accepts_dfa(dfa1par, '101001'))
print("accepts_dfa(dfa0par, '101001') = ", 
          accepts_dfa(dfa0par, '101001'))
print("accepts_dfa(dfa0ou1par, '101001') = ", 
          accepts_dfa(dfa0ou1par, '101001'))


accepts_dfa(dfa1par, '10100') =  True
accepts_dfa(dfa0par, '10100') =  False
accepts_dfa(dfa0ou1par, '10100') =  True

accepts_dfa(dfa1par, '101001') =  False
accepts_dfa(dfa0par, '101001') =  False
accepts_dfa(dfa0ou1par, '101001') =  False


## Totalização de um autómato

In [ ]:
def totalize_dfa(D):

    assert(partconsist_dfa(D)), "DFA is not partially consistent."

    new_Delta = D["Delta"].copy()
    
    gaps = { (q,c) : "BH" for q in D["Q"] for c in D["Sigma"] if (q,c) not in D["Delta"] }

    new_Delta.update( gaps )
    
    bh_self = { ("BH", c): "BH" for c in D["Sigma"] }

    new_Delta.update( bh_self )
        
    return {"Q"    : D["Q"] | { "BH" }, 
            "Sigma": D["Sigma"],    
            "Delta": new_Delta,
            "q0"   : D["q0"],          
            "F"    : D["F"] }


In [ ]:
dfax = { 'Q': {'A', 'B'},
          'Sigma': {'0', '1'},
          'Delta': { ('A', '1'): 'B',
                     ('B', '1'): 'A' },
          'q0': 'A', 
          'F': {'A'}   
        }

print(consist_dfa(dfax))

dfaxt = totalize_dfa(dfax)
print(dfaxt)
print(consist_dfa(dfaxt))


False
{'Q': {'BH', 'B', 'A'}, 'Sigma': {'1', '0'}, 'Delta': {('A', '1'): 'B', ('B', '1'): 'A', ('B', '0'): 'BH', ('A', '0'): 'BH', ('BH', '1'): 'BH', ('BH', '0'): 'BH'}, 'q0': 'A', 'F': {'A'}}
True
